In [3]:
import requests
import sqlite3
from google.transit import gtfs_realtime_pb2

In [4]:
def parse_feed(feed):
    vehicle_positions = []
    for entity in feed.entity:
        if entity.HasField('vehicle'):
            vehicle = entity.vehicle
            vehicle_positions.append({
                'vehicle_id': vehicle.vehicle.id,
                'trip_id': vehicle.trip.trip_id,
                'route_id': vehicle.trip.route_id,
                'schedule_relationship': vehicle.trip.schedule_relationship,
                'latitude': vehicle.position.latitude,
                'longitude': vehicle.position.longitude,
                'bearing': vehicle.position.bearing,
                'speed': vehicle.position.speed,
                'timestamp': vehicle.timestamp,
                'occupancy_status': str(vehicle.occupancy_status)
            })
    return vehicle_positions

In [ ]:
def insert_trips_table(cursor, record):
    cursor.execute('''
    INSERT INTO trips (
        trip_id, route_id, timestamp, schedule_relationship, vehicle_id
    ) VALUES (?, ?, ?, ?, ?)
    ''', (
        record['trip_id'], record['route_id'], 
        record['timestamp'], record['schedule_relationship'], 
        record['vehicle_id']
    ))

In [ ]:
def insert_vehicle_positions_table(cursor, record):
    cursor.execute('''
    INSERT INTO vehicle_positions (
        vehicle_id, timestamp, latitude, 
        longitude, bearing, speed, occupancy_status
    ) VALUES (?, ?, ?, ?, ?, ?, ?)
    ''', (
        record['vehicle_id'], record['timestamp'], record['latitude'], 
        record['longitude'], record['bearing'], record['speed'], 
        record['occupancy_status']
    ))

In [6]:
# Parse realtime protobuf feed
feed = gtfs_realtime_pb2.FeedMessage()
response = requests.get('https://bustime.ttc.ca/gtfsrt/vehicles')
feed.ParseFromString(response.content)

import pprint

vehicle_positions = parse_feed(feed)
pprint.pprint(vehicle_positions)
# print(vehicle_positions)
print(len(vehicle_positions))

[{'bearing': 347.0,
  'latitude': 43.775638580322266,
  'longitude': -79.34680938720703,
  'occupancy_status': '0',
  'route_id': '85',
  'schedule_relationship': 0,
  'speed': 0.0,
  'timestamp': 1749522578,
  'trip_id': '19417020',
  'vehicle_id': '1220'},
 {'bearing': 255.0,
  'latitude': 43.725830078125,
  'longitude': -79.4861068725586,
  'occupancy_status': '2',
  'route_id': '96',
  'schedule_relationship': 0,
  'speed': 0.44703999161720276,
  'timestamp': 1749522582,
  'trip_id': '124646020',
  'vehicle_id': '3639'},
 {'bearing': 77.0,
  'latitude': 43.762733459472656,
  'longitude': -79.41322326660156,
  'occupancy_status': '0',
  'route_id': '',
  'schedule_relationship': 0,
  'speed': 0.0,
  'timestamp': 1749522578,
  'trip_id': '',
  'vehicle_id': '1215'},
 {'bearing': 152.0,
  'latitude': 43.79366683959961,
  'longitude': -79.24317932128906,
  'occupancy_status': '0',
  'route_id': '',
  'schedule_relationship': 0,
  'speed': 0.0,
  'timestamp': 1749522583,
  'trip_id': ''

In [69]:
conn = sqlite3.connect('../rtds-ttc.db')
cursor = conn.cursor()

for record in vehicle_positions:
    insert_vehicle_positions_table(cursor, record)

conn.commit()
conn.close()
